In [1]:
# import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from fuzzywuzzy import fuzz
import sqlite3
from sqlite3 import Error

assets_path = '../data/'

citybikes = pd.read_csv(assets_path + 'citybikes.csv')
fs_data = pd.read_csv(assets_path + 'fs_data.csv')
yelp_data = pd.read_csv(assets_path + 'yelp_data.csv')


In [2]:
fs_data = fs_data.reindex(columns=['fsq_id', 'category_id', 'name', 'latitude', 'longitude', 'location_formatted_address', 'bike_station_id', 'distance'] )
fs_data.sort_values(['bike_station_id','name','latitude','longitude', 'distance']).head(10)

,fsq_id,category_id,name,latitude,longitude,location_formatted_address,bike_station_id,distance
400,4b7205ecf964a520ac6a2de3,13065,A&W Restaurant,49.260739,-123.116314,"555 12th Ave W, Vancouver BC V5Z 3X7",07b13214cb69f2ec007d888450f3d6bc,179
415,94e7b9924728493cb0bad9de,13065,Arby's Roast Beef Restaurants,49.260760,-123.116397,"555 12th Ave W, Vancouver BC V5Z 3X7",07b13214cb69f2ec007d888450f3d6bc,176
406,5b6c8e6bbd4009002c2b3bcf,13065,Bubble Waffle Cafe,49.260909,-123.116328,"555 12th Ave W, Vancouver BC V5Z 3X7",07b13214cb69f2ec007d888450f3d6bc,174
405,6194486bcfc5a57be287e71f,13065,Chirpyhut Fried Chicken,49.258082,-123.114546,"3088 Cambie St, Vancouver BC V5Z 2V9",07b13214cb69f2ec007d888450f3d6bc,164
413,80d53a43efc54e3eb840f27b,13065,Crepe de France,49.260646,-123.116387,"555 12th Ave W, Vancouver BC V5Z 3X7",07b13214cb69f2ec007d888450f3d6bc,166
409,4be48099910020a134ced114,13065,Cultured Coffee & Tea,49.260796,-123.116318,555 12th Ave W (at Cambie St in City Square Sh...,07b13214cb69f2ec007d888450f3d6bc,189
416,1c177002d1304ae7f9f71ea2,13065,Edo Japan-edo Intl Food Inc,49.260693,-123.116796,"555 12th Ave W, Vancouver BC V5Z 3X7",07b13214cb69f2ec007d888450f3d6bc,192
407,62d47a8219f50c000af52313,13065,Five Star Fried Chicken,49.260909,-123.116328,"555 12th Ave W (Cambie St), Vancouver BC V5Z 3X7",07b13214cb69f2ec007d888450f3d6bc,179
403,4bc2aa604cdfc9b6a9749621,13065,Hot Delivery Chinese Food,49.258597,-123.114833,"3002 Cambie St, Vancouver BC V5Z 2V9",07b13214cb69f2ec007d888450f3d6bc,111
395,52ae4f3c11d227ba4f4b055a,13065,Indian Roti Kitchen,49.258983,-123.115272,"2961 Cambie St, Vancouver BC V5Z 2V7",07b13214cb69f2ec007d888450f3d6bc,73


In [3]:
yelp_data = yelp_data.reindex(columns=['yelp_id',  'category_id', 'name', 'latitude', 'longitude', 'rating', 'review_count', 'bike_station_id', 'distance'] )
yelp_data.sort_values(['bike_station_id','name','latitude','longitude', 'distance']).head(10)


,yelp_id,category_id,name,latitude,longitude,rating,review_count,bike_station_id,distance
332,a2TPk6l9bnsXHjIwjZt0nw,13065,A&W,49.260860,-123.116290,2.0,10,07b13214cb69f2ec007d888450f3d6bc,200
324,NAP1sxQaytZs9HPAP9o5cQ,13065,Bubble Waffle Cafe,49.260860,-123.116290,4.0,17,07b13214cb69f2ec007d888450f3d6bc,200
341,jHJrCb-MgilhZoLF1VV7nQ,13065,CanTresor Pretzels,49.260860,-123.116290,2.0,2,07b13214cb69f2ec007d888450f3d6bc,200
334,KZCG_rFrSixuGCmQWhghOQ,13065,Chili Tank,49.261226,-123.113927,5.0,1,07b13214cb69f2ec007d888450f3d6bc,193
326,AdlGyoE9ff7dxZv-tklsLQ,13065,Chirpyhut Fried Chicken,49.258020,-123.114915,3.5,13,07b13214cb69f2ec007d888450f3d6bc,165
342,UHoaUfR1gWsSo3Nr6_5nBg,13065,Dairy Queen,49.260834,-123.116287,0.0,0,07b13214cb69f2ec007d888450f3d6bc,177
331,nVCUL9tCURfWGAJIw0F5Fw,13065,Five Star Fried Chicken,49.260860,-123.116290,3.0,6,07b13214cb69f2ec007d888450f3d6bc,200
330,B_Fh6c4ltuglgTsSGUwmVA,13065,Freshii,49.260988,-123.116495,3.5,3,07b13214cb69f2ec007d888450f3d6bc,200
336,1EmTFvc3EUWPHuVt_9UB0w,13065,Hiro Sushi,49.260860,-123.116287,4.0,1,07b13214cb69f2ec007d888450f3d6bc,164
321,pQsTrwmMl9qRlXaJ9_RCzw,13065,Hot Delivery Chinese Food,49.258697,-123.114943,2.5,54,07b13214cb69f2ec007d888450f3d6bc,98


In [4]:
yelp_data['yelp_id'].nunique() == len(yelp_data)

True

In [5]:
fs_data['fsq_id'].nunique() == len(fs_data)


True

In [6]:
def similarity(s1, s2):
    return fuzz.token_set_ratio(s1, s2)

merged_arr = []

for i, row1 in fs_data.iterrows():
    for j, row2 in yelp_data.iterrows():
        if similarity(row1['name'], row2['name']) >= 80 \
            and abs(row1['latitude'] - row2['latitude']) <= 0.0001 \
            and abs(row1['longitude'] - row2['longitude']) <= 0.0001:
                merged_arr.append({
                    'name':row2['name'],
                    'category_id':int(row2['category_id']),
                    'latitude':row2['latitude'],
                    'longitude':row2['longitude'],
                    'location_formatted_address':row1['location_formatted_address'],
                    'rating':row2['rating'],
                    'review_count':row2['review_count'],
                    'bike_station_id':row2['bike_station_id'],
                    'distance': (row1['distance'] + row2['distance'])/2
                })

restaurants = pd.DataFrame(merged_arr)
restaurants.describe()

,category_id,latitude,longitude,rating,review_count,distance
count,117.0,117.000000,117.000000,117.000000,117.000000,117.000000
mean,13065.0,49.274856,-123.116520,3.589744,136.837607,124.076923
std,0.0,0.011596,0.025560,0.715034,165.261076,42.610284
min,13065.0,49.237413,-123.165923,1.500000,1.000000,10.500000
25%,13065.0,49.267382,-123.131742,3.000000,31.000000,89.500000
50%,13065.0,49.277787,-123.120450,3.500000,85.000000,130.000000
75%,13065.0,49.283782,-123.103958,4.000000,181.000000,154.000000
max,13065.0,49.290680,-123.062273,5.000000,987.000000,199.000000


In [7]:
restaurants.to_csv(assets_path+"restaurants.csv", index=False)
restaurants.head()

,name,category_id,latitude,longitude,location_formatted_address,rating,review_count,bike_station_id,distance
0,Congee Noodle Delight,13065,49.280590,-123.106225,"88 Pender St W, Vancouver BC V6B 6N9",3.5,91,97b25faf822b93fdbf72da1d594c0288,142.0
1,Tagine Moroccan Restaurant and Bar,13065,49.281980,-123.106330,"51 Hastings St W, Vancouver BC V6B 1G4",4.0,148,97b25faf822b93fdbf72da1d594c0288,162.5
2,Caveman Cafe,13065,49.280802,-123.107035,"88 Pender St W (Abbott St), Vancouver BC V6B 6N9",4.5,270,97b25faf822b93fdbf72da1d594c0288,195.0
3,Kouign Café,13065,49.280313,-123.103958,"18 Pender St E, Vancouver BC V6A 1T1",4.5,36,97b25faf822b93fdbf72da1d594c0288,89.5
4,Adriatic Divine Sandwiches,13065,49.281616,-123.102443,"375 Columbia St, Vancouver BC",5.0,1,97b25faf822b93fdbf72da1d594c0288,152.0


In [8]:
citybikes.head()

,Free bikes,Latitude,Longitude,Empty slots,Id
0,18,49.281170,-123.104419,0,97b25faf822b93fdbf72da1d594c0288
1,8,49.294318,-123.148808,10,9aa325529164ec68b4d96c4f1c4fc7c1
2,10,49.298657,-123.121176,10,e3d32ad772a5898c3b96c4d23d2cd542
3,7,49.258924,-123.152591,13,cadc004f0903ef45e898032143c0832f
4,12,49.266774,-123.141135,4,57e928d8c30fab0ebffb3ede850f89d6


In [9]:
import sqlite3
from sqlite3 import Error

folder = '../data/'
db_path = folder + "StatModeling.db"

try:
    conn = sqlite3.connect(db_path, timeout=10)
    print(f"Connected to SQLite database at {db_path}")

except Error as e:
    print(e)

create_bike_stations_restaurants = """
 CREATE TABLE IF NOT EXISTS bike_stations_restaurants (
     restaurant_id INTEGER NOT NULL,
     bike_station_id VARCHAR(30) NOT NULL,
     distance NUMERIC(4,1) NOT NULL,
     PRIMARY KEY(restaurant_id, bike_station_id),
     FOREIGN KEY (bike_station_id) REFERENCES bike_stations(bike_stations_id),
     FOREIGN KEY (restaurant_id) REFERENCES restaurants(restaurant_id)
 );
 """

create_restaurants = """
    CREATE TABLE IF NOT EXISTS restaurants (
        restaurant_id INTEGER PRIMARY KEY,
        name VARCHAR(50) NOT NULL,
        category_id INT8 NOT NULL,
        latitude NUMERIC(9,6) NOT NULL,
        longitude NUMERIC(9,6) NOT NULL,
        location_formatted_address TEXT NOT NULL,
        rating NUMERIC(2,1) NOT NULL,
        review_count INT16 NOT NULL
);

    """

create_bike_stations = """
    CREATE TABLE IF NOT EXISTS bike_stations (
        bike_stations_id VARCHAR(30) PRIMARY KEY,
        latitude NUMERIC(9,6) NOT NULL,
        longitude NUMERIC(9,6) NOT NULL,
        free_bikes INT8 NOT NULL,
        empty_slots INT8 NOT NULL
);
    """

def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        # print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as e:
        print(f"The error '{e}' occurred")

execute_query(conn,"""DROP TABLE restaurants""")
execute_query(conn,"""DROP TABLE bike_stations""")
execute_query(conn,"""DROP TABLE bike_stations_restaurants""")


execute_query(conn,create_bike_stations)

for index, row in citybikes.iterrows():
    query=f"""INSERT INTO bike_stations VALUES("{row['Id']}",{row['Latitude']},{row['Longitude']},{row['Free bikes']},{row['Empty slots']})"""
    execute_query(conn,query)


execute_query(conn,create_restaurants)
execute_query(conn,create_bike_stations_restaurants)


for index, row in restaurants.iterrows():
    new_id = None
    try:
        cursor = conn.cursor()
        cursor.execute(f"""
            INSERT INTO restaurants 
                (name,
                category_id,
                latitude,
                longitude,
                location_formatted_address,
                rating,
                review_count)
            VALUES("{row['name']}"
                ,{row['category_id']}
                ,{row['latitude']}
                ,{row['longitude']}
                ,"{row['location_formatted_address']}"
                ,{row['rating']}
                ,{row['review_count']}
            )""")
        new_id = cursor.lastrowid
    except Error as e:
        print(f"The error '{e}' occurred")


    if new_id:
        query=f"""
            INSERT INTO bike_stations_restaurants(
                restaurant_id,
                bike_station_id,
                distance
            ) 
            VALUES({new_id}
                ,"{row['bike_station_id']}"
                ,{row['distance']}
            )""" 

        execute_query(conn,query)


Connected to SQLite database at ../data/StatModeling.db
The error 'no such table: bike_stations_restaurants' occurred


Join the data from Part 1 with the data from Part 2 to create a new dataframe.

Provide a visualization that you used as part of your EDA process. Explain the initial pattern or relationship you discoved through this visualization. 

# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

Look at the data before and after the join to validate your data.